In [16]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.LocalSession()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# S3 prefix
#bucket = sagemaker_session.default_bucket()
dependancy_path = "dependencies.py"
train_data = "s3://empid-bucketname/housing_train_data.csv"

In [17]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = "sklearn_entry_point_Latest.py"

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    dependencies=['dependencies.py'],
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type="local",
    sagemaker_session=sagemaker_session,
)

In [18]:
sklearn_preprocessor.fit({"train": train_data})

Creating 70t9s180qs-algo-1-441vb ... 
Creating 70t9s180qs-algo-1-441vb ... done
Attaching to 70t9s180qs-algo-1-441vb
70t9s180qs-algo-1-441vb | 2021-05-20 06:35:32,873 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
70t9s180qs-algo-1-441vb | 2021-05-20 06:35:32,877 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
70t9s180qs-algo-1-441vb | 2021-05-20 06:35:32,889 sagemaker_sklearn_container.training INFO     Invoking user training script.
70t9s180qs-algo-1-441vb | 2021-05-20 06:35:33,063 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
70t9s180qs-algo-1-441vb | 2021-05-20 06:35:33,084 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
70t9s180qs-algo-1-441vb | 2021-05-20 06:35:33,099 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
70t9s180qs-algo-1-441vb | 2021-05-20 06:35:33,111 sagemaker-training-toolkit INFO     In

In [19]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, instance_type="local", assemble_with="Line", accept="text/csv"
)

In [20]:
# Preprocess training input
transformer.transform(train_data, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
#output_path="s3://empid-bucketname/trnasformer_train_data.csv"
transformer.wait()
preprocessed_train = transformer.output_path

Attaching to y9fh7fwsqv-algo-1-jwbgm
y9fh7fwsqv-algo-1-jwbgm | 2021-05-20 06:36:06,830 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
y9fh7fwsqv-algo-1-jwbgm | 2021-05-20 06:36:06,833 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
y9fh7fwsqv-algo-1-jwbgm | 2021-05-20 06:36:06,834 INFO - sagemaker-containers - nginx config: 
y9fh7fwsqv-algo-1-jwbgm | worker_processes auto;
y9fh7fwsqv-algo-1-jwbgm | daemon off;
y9fh7fwsqv-algo-1-jwbgm | pid /tmp/nginx.pid;
y9fh7fwsqv-algo-1-jwbgm | error_log  /dev/stderr;
y9fh7fwsqv-algo-1-jwbgm | 
y9fh7fwsqv-algo-1-jwbgm | worker_rlimit_nofile 4096;
y9fh7fwsqv-algo-1-jwbgm | 
y9fh7fwsqv-algo-1-jwbgm | events {
y9fh7fwsqv-algo-1-jwbgm |   worker_connections 2048;
y9fh7fwsqv-algo-1-jwbgm | }
y9fh7fwsqv-algo-1-jwbgm | 
y9fh7fwsqv-algo-1-jwbgm | http {
y9fh7fwsqv-algo-1-jwbgm |   include /etc/nginx/mime.types;
y9fh7fwsqv-algo-1-jwbgm |   default_type application/octet-stream;
y9fh7fwsqv-algo-1-jwbg

In [21]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()


model_name = "inference-" + timestamp_prefix
endpoint_name = "inference-ep-" + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, role=role, models=[scikit_learn_inferencee_model]
)

sm_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge", endpoint_name=endpoint_name)

---------------!

In [27]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, Predictor
#from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

csv_serializer = sagemaker.serializers.CSVSerializer(content_type="text/csv")
data = """-122.26,37.46,26,5067,750,1996,728,7.0001,NEAR OCEAN"""

sagemaker_session = sagemaker.Session()

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=csv_serializer)


print(predictor.predict(data))

b'{"instances": [{"features": [-1.3408764930120565, 0.8514634482951091, -0.21099315585118414, 1.1430410231864847, 0.5257730939190086, 0.5164773116405509, 0.6146760713962672, 1.6402561279658028, 0.5819441198401629, -0.030616763948385764, -1.0052566158906469, 0.0, 0.0, 0.0, 0.0, 1.0]}]}'
